# DS-SF-25 | Codealong 12 | Decision Trees and Random Forests

In [1]:
import os

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

import math

## Part A - The 2008 Democratic Primaries

(dataset adapted from http://www.stat.ucla.edu/~cocteau/primaries.csv)

In [16]:
df = pd.read_csv(os.path.join('..', 'datasets', '2008-democrat-primaries.csv'))

In [17]:
df.columns

Index([u'fips', u'county_name', u'state_postal', u'region', u'election_date',
       u'racetype', u'tvotes', u'clinton', u'obama', u'edwards', u'margin',
       u'winner', u'POP05_SQMI', u'popUnder30_00', u'pop65up_00',
       u'presVote04', u'kerry04', u'Bush04', u'pres04margin', u'pres04winner',
       u'pop06', u'pop00', u'hisp06', u'white06', u'black06', u'indian06',
       u'asian06', u'hawaii06', u'mixed06', u'pct_less_30k', u'pct_more_100k',
       u'pct_hs_grad', u'pct_labor_force', u'pct_homeowner', u'unempFeb07',
       u'unempFeb08', u'unempChg', u'pctUnins00', u'subForPctHomes',
       u'poverty05', u'median_hhi05', u'Catholic', u'So.Bapt.Conv',
       u'Un.Methodist', u'E.L.C.A.', u'Construction', u'Manufacturing',
       u'FinancialActivities', u'GoodsProducing', u'ServiceProviding'],
      dtype='object')

In [14]:
df.winner.value_counts()

clinton    1210
obama      1031
Name: winner, dtype: int64

In [24]:
df['c']=df.winner
df.c

0         obama
1       clinton
2         obama
3       clinton
4       clinton
         ...   
2256    clinton
2257      obama
2258      obama
2259      obama
2260    clinton
Name: c, dtype: object

### First cut: Is a county more than 20% black?

In [22]:
df['perc_black_over20']=df.black06*1./df.pop06>0.2
df.perc_black_over20.value_counts()

False    1811
True      450
Name: perc_black_over20, dtype: int64

In [53]:
parent_df=df
right_child_df=df[df.perc_black_over20==True]
left_child_df=parent_df.drop(right_child_df.index)


In [44]:
print (right_child_df.c=='obama').sum()
print (right_child_df.c=='clinton').sum()

380
69


In [45]:
print (left_child_df.c=='obama').sum()
print (left_child_df.c=='clinton').sum()

651
1141


In [54]:
parent_df=left_child_df
right_child_df=parent_df[parent_df.pct_hs_grad>0.78]
left_child_df=parent_df.drop(right_child_df.index)


In [55]:
print (right_child_df.c=='obama').sum()
print (right_child_df.c=='clinton').sum()

556
433


In [56]:
print (left_child_df.c=='obama').sum()
print (left_child_df.c=='clinton').sum()

95
708


In [59]:
def predict(black,educated):
    if black is True:
        return 'Obama'
    if eductated is True:
        return 'Obama'
    else:
        return 'Clinton'
predict(True,True)

'Obama'

In [73]:
def predict(X):
    if X[0] is True:
        return 'Yes'
    if X[1] is True:
        return 'Yes'
    else:
        return 'No'

X=['True','True']

predict(X)

'No'

1. set root node to parent node
2. while parent node is not a leaf node
    evaluate split condition of the parent node using x
         if true set left node as new parent node
         otherwise set right node as new parent node
       repeat
3. return class of leaf leaf node

#### First cut/right node

In [ ]:
# TODO

In [74]:
def obama_vs_clinton(df):
    obama = (df.c == 1).sum()
    clinton = (df.c == 0).sum()
    if obama > clinton:
        print 'Obama wins these counties {} to {}.'.format(obama, clinton)
    elif clinton > obama:
        print 'Clinton wins these counties {} to {}.'.format(clinton, obama)
    else:
        print 'Obama and Clinton tie in these counties {} {}.'.format(obama, clinton)

In [75]:
obama_vs_clinton(right_child_df)

Obama and Clinton tie in these counties 0 0.


### Second cut: Is high school graduation rate higher than 78%?

In [ ]:
# TODO

In [ ]:
obama_vs_clinton(left_child_df)

### Third cut: Is high school graduation rate higher than 87%?

In [ ]:
# TODO

In [ ]:
obama_vs_clinton(right_child_df)

## Part B - Building the 2008 Democratic Primaries Decision Tree by Hand

In [ ]:
class Node:

    @staticmethod
    def root(root_df):
        cs = sorted(set(root_df.c))
        return Node(cs, root_df)

    def decision(self, left_filter):
        # Collect the observations for which the decision split is true and
        # create the corresponding left node

        left_filter = left_filter(self.df)
        left_df = self.df[left_filter]
        self.left = Node(self.cs, left_df)

        # Same thing on the right side but for the observations that don't
        # satisfy the decision split (the "else")

        right_df = self.df.drop(left_df.index)
        self.right = Node(self.cs, right_df)

        # The entropy after the decision split is the weighted average of the
        # children entropy

        self.after = (self.left.samples * self.left.before
                      + self.right.samples * self.right.before) / self.samples

        # The information gain corresponds to the entropy lost between the
        # parent node (this node and the "before") and its child (the "after")

        self.information_gain = self.before - self.after

        return self

    def __init__(self, cs, df):
        self.cs = cs
        self.df = df

        # Counts of the remaining observations in the subspace per classes
        self.counts = [sum(self.df.c == c) for c in self.cs]

        # Number of observations in the subspace
        self.samples = sum(self.counts)

        # For empty subspaces, probabilties and entropy are set to zero
        if self.samples == 0:
            self.probabilities = [0. for count in self.counts]
            self.before = 0.
        else:
            self.probabilities = [1. * count / self.samples for count in self.counts]
            self.before = - sum(map(lambda p: p * math.log(p, 2),
                                    filter(lambda p : p > 0., self.probabilities)))

    def status(self):
        print 'classes                       =', self.cs
        print 'before:'
        print "\tparent:"
        print "\t\tsamples       =", self.samples
        print "\t\tcounts        =", self.counts
        print "\t\tprobabilities =", self.probabilities
        print "\t\tentropy       =", self.before
        print 'after:'
        print "\tleft child:"
        print "\t\tsamples       =", self.left.samples
        print "\t\tcounts        =", self.left.counts
        print "\t\tprobabilities =", self.left.probabilities
        print "\t\tentropy       =", self.left.before
        print "\tright child:"
        print "\t\tsamples       =", self.right.samples
        print "\t\tcounts        =", self.right.counts
        print "\t\tprobabilities =", self.right.probabilities
        print "\t\tentropy       =", self.right.before
        print
        print 'before entropy                =', self.before
        print 'after entropy                 =', self.after
        print 'information gain              =', self.information_gain

In [ ]:
df.c = df.winner

### First cut

In [ ]:
# TODO

#### Candidate #1: Is a county more than 20% black?

In [ ]:
# TODO

#### Candidate #2: Is high school graduation rate higher than 78%?

In [ ]:
# TODO

#### Candidate #3: Is high school graduation rate higher than 87%?

In [ ]:
# TODO

### Second cut

In [ ]:
# TODO

### Third cut

In [ ]:
# TODO